# Download dataset


In [ ]:
dataset_dir = '/content/isaid_small_cropped'

In [ ]:
%cd /content

/content


In [ ]:
import requests
import os
from urllib.parse import urlencode


def download_yd_file(public_key: str, save_path: str):

    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    print(response.status_code)
    download_url = response.json()['href']

    download_response = requests.get(download_url)
    with open(save_path, 'wb') as f:
        f.write(download_response.content)

In [ ]:
links = {
    'isaid_small.zip': 'https://disk.yandex.ru/d/6KHSKFgIonskuA',#'https://disk.yandex.ru/d/qPlkJoQCwcZGow',
}

for filename in links.keys():
    src_link = links[filename]
    dst_path = os.path.join('/content', filename)
    print(dst_path)
    download_yd_file(src_link, dst_path)

/content/isaid_small.zip
200


In [ ]:
!unzip -q isaid_small.zip -d /content

#Train

In [ ]:
%cd /content/drive/MyDrive

import os

model = 'yolov8s-seg.pt'

epochs = 10
imgsz = 640
batch = 8
warmup = 3
half = True
lr0 = 0.01

copy_paste = 0.1
mixup = 0.1
degrees = 25
flipud = 0.5


name = 'yolov8s_seg_isaid_2504'
data = os.path.join(dataset_dir, 'data.yaml')

!yolo segment train \
    data={data} \
    model={model} \
    epochs={epochs} \
    imgsz={imgsz} \
    batch={batch} \
    warmup_epochs={warmup} \
    half={half} \
    lr0={lr0} \
    copy_paste={copy_paste} \
    mixup={mixup} \
    degrees={degrees} \
    flipud={flipud} \
    name={name}

/content/drive/MyDrive
Ultralytics YOLOv8.0.87 🚀 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/content/isaid_small/data.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8s_seg_isaid_2504, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=Fal

# Validate

In [ ]:
# Write your model path
model = '/content/drive/MyDrive/runs/segment/yolov8sseg_isaid3/weights/best.pt'

name = 'yolov8sseg_isaid'
data = os.path.join(dataset_dir, 'data.yaml')

!yolo segment val model={model} name={name} data={data}

Ultralytics YOLOv8.0.87 🚀 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-seg summary (fused): 195 layers, 11785405 parameters, 0 gradients, 42.5 GFLOPs
100% 755k/755k [00:00<00:00, 40.2MB/s]
val: Scanning /content/isaid_small/valid/labels... 45 images, 0 backgrounds, 0 corrupt: 100% 45/45 [00:01<00:00, 23.12it/s]
val: New cache created: /content/isaid_small/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% 3/3 [00:22<00:00,  7.51s/it]
                   all         45       7984      0.593     0.0688     0.0488     0.0322      0.587     0.0675     0.0489     0.0306
          storage_tank         45          1          1          0          0          0          1          0          0          0
         Large_Vehicle         45        609      0.334      0.374       0.27      0.116      0.311      0.353      0.249      0.107
         Small_Vehicle         45

# Predict

In [ ]:
import cv2
import wandb

wandb.init(project='YOLOv8')

In [ ]:
import cv2
from PIL import Image

from google.colab.patches import cv2_imshow
from ultralytics import YOLO

# Load the YOLOv8 model
model = '/content/drive/MyDrive/runs/segment/yolov8s_seg_isaid_25044/weights/best.pt'
model = YOLO(model)

valid_images_dir = os.path.join(dataset_dir, 'valid', 'images')
valid_image_files = os.listdir(valid_images_dir)
valid_image_files.sort()
valid_image_files = valid_image_files[:10]

for f in valid_image_files:
    img = cv2.imread(os.path.join(valid_images_dir, f))
    res = model(img)
    res_plotted = res[0].plot()
    #cv2_imshow(res_plotted)

    #res_plotted = Image.fromarray(res_plotted)
    wandb.log({f'{f} ': wandb.Image(res_plotted)})




0: 640x640 19 Large_Vehicles, 22.8ms
Speed: 1.9ms preprocess, 22.8ms inference, 5.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x512 3 Large_Vehicles, 5 Small_Vehicles, 22.1ms
Speed: 1.7ms preprocess, 22.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 Large_Vehicles, 3 Small_Vehicles, 2 Harbors, 14.9ms
Speed: 1.4ms preprocess, 14.9ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 512x640 6 Large_Vehicles, 6 Small_Vehicles, 17.6ms
Speed: 4.4ms preprocess, 17.6ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 36 Large_Vehicles, 4 Small_Vehicles, 25.9ms
Speed: 2.5ms preprocess, 25.9ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x192 5 Small_Vehicles, 14.6ms
Speed: 1.7ms preprocess, 14.6ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 29 Large_Vehicles, 6 Small_Vehicles, 21.9ms
Speed: 2.0ms preprocess, 21.9ms inference, 3.6ms postpr

In [ ]:
wandb.finish()